# VacationPy
----

####  Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [5]:
cities_df = pd.read_csv("./output_data/cities.csv")
cities_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,barrow,71.2906,-156.7887,28.42,93,90,18.41,US,1623076122
1,1,pisco,-13.7000,-76.2167,64.45,77,90,3.44,PE,1623076452
2,2,arraial do cabo,-22.9661,-42.0278,76.96,78,40,11.50,BR,1623076452
3,3,kapaa,22.0752,-159.3190,75.52,81,75,7.00,US,1623076453
4,4,saint-philippe,-21.3585,55.7679,72.99,76,87,16.08,RE,1623076453


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

In [11]:
# Store 'Lat' and 'Lng' into  locations 
locations = cities_df[["Lat", "Lng"]].astype(float)
humidity = cities_df["Humidity"].astype(float)

In [12]:
fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [33]:
ideal_weather_df = pd.DataFrame(cities_df, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness","Country", "Lat", "Lng"])

max_temp = (ideal_weather_df["Max Temp"] <= 80) & (ideal_weather_df["Max Temp"] > 70)
wind_speed = ideal_weather_df["Wind Speed"] < 10
cloudiness = ideal_weather_df["Cloudiness"] == 0

ideal_weather_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness,Country,Lat,Lng
99,opuwo,78.35,9.69,0,NaN,-18.0607,13.8400
212,dunkirk,76.57,7.58,0,FR,51.0500,2.3667
221,port alfred,71.04,9.93,0,ZA,-33.5906,26.8910
234,beauvais,73.72,1.01,0,FR,49.4167,2.0000
276,dir,77.65,2.13,0,PK,35.2058,71.8756
488,evreux,73.92,3.44,0,FR,49.0241,1.1508


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
#Store into variable named hotel_df
hotel_df = ideal_weather_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,barrow,US,71.2906,-156.7887,
1,pisco,PE,-13.7000,-76.2167,
2,arraial do cabo,BR,-22.9661,-42.0278,
3,kapaa,US,22.0752,-159.3190,
4,saint-philippe,RE,-21.3585,55.7679,
...,...,...,...,...,...
558,meulaboh,ID,4.1363,96.1285,
559,coffs harbour,AU,-30.3000,153.1333,
560,muyuka,CM,4.2898,9.4103,
561,contamana,PE,-7.3333,-75.0167,


In [35]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [36]:
import time

In [37]:
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 0: barrow.
Closest hotel in barrow is Top of the World Hotel.
--------------------Ending Search
Retrieving Results for Index 1: pisco.
Closest hotel in pisco is REGENCY PLAZA HOTEL.
--------------------Ending Search
Retrieving Results for Index 2: arraial do cabo.
Closest hotel in arraial do cabo is Ocean View Hotel.
--------------------Ending Search
Retrieving Results for Index 3: kapaa.
Closest hotel in kapaa is Kauai Shores Hotel.
--------------------Ending Search
Retrieving Results for Index 4: saint-philippe.
Closest hotel in saint-philippe is Les Embruns Du Baril.
--------------------Ending Search
Retrieving Results for Index 5: wuan.
Closest hotel in wuan is Xinyuan Hotel.
--------------------Ending Search
Retrieving Results for Index 6: almaznyy.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 7: vardo.
Closest hotel in vardo is Vardø Hotel AS.
--------------------Ending Search
Retrievin

Closest hotel in sao jose da coroa grande is Pousada Vivenda Oriente.
--------------------Ending Search
Retrieving Results for Index 65: lagoa.
Closest hotel in lagoa is Graciosa Resort - Biosphere Island Hotel.
--------------------Ending Search
Retrieving Results for Index 66: kholodnyy.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 67: hobart.
Closest hotel in hobart is MACq 01 Hotel.
--------------------Ending Search
Retrieving Results for Index 68: souillac.
Closest hotel in souillac is SkylineInn Hotel.
--------------------Ending Search
Retrieving Results for Index 69: deputatskiy.
Closest hotel in deputatskiy is Sberbank.
--------------------Ending Search
Retrieving Results for Index 70: lerwick.
Closest hotel in lerwick is The Grand Hotel.
--------------------Ending Search
Retrieving Results for Index 71: qaanaaq.
Closest hotel in qaanaaq is Qaanaaq Hotel.
--------------------Ending Search
Retrieving Results for In

Closest hotel in maloy is Thon Hotel Måløy.
--------------------Ending Search
Retrieving Results for Index 128: madang.
Closest hotel in madang is Madang Resort.
--------------------Ending Search
Retrieving Results for Index 129: castro.
Closest hotel in castro is Central Palace Hotel - Castro (PR).
--------------------Ending Search
Retrieving Results for Index 130: chokurdakh.
Closest hotel in chokurdakh is Sberbank.
--------------------Ending Search
Retrieving Results for Index 131: yellowknife.
Closest hotel in yellowknife is The Explorer Hotel.
--------------------Ending Search
Retrieving Results for Index 132: hilo.
Closest hotel in hilo is Hilo Hawaiian Hotel.
--------------------Ending Search
Retrieving Results for Index 133: grand forks.
Closest hotel in grand forks is TLF Temporary Lodging Facility.
--------------------Ending Search
Retrieving Results for Index 134: nyurba.
Closest hotel in nyurba is Gostinitsa Alrosa.
--------------------Ending Search
Retrieving Results for I

--------------------Ending Search
Retrieving Results for Index 191: payson.
Closest hotel in payson is Quality Inn Payson I-15.
--------------------Ending Search
Retrieving Results for Index 192: labuhan.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 193: valleyview.
Closest hotel in valleyview is Paradise Inn & Suites.
--------------------Ending Search
Retrieving Results for Index 194: chaah.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 195: umm kaddadah.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 196: bratsk.
Closest hotel in bratsk is Гостевой дом "Времена года".
--------------------Ending Search
Retrieving Results for Index 197: srandakan.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 198: khatanga.
Closest hotel in khatan

Closest hotel in valdivia is Hotel Naguilan.
--------------------Ending Search
Retrieving Results for Index 256: nuevo vigia.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 257: dingle.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 258: the valley.
Closest hotel in the valley is The Manoah Boutique Hotel | Shoal Bay - Anguilla.
--------------------Ending Search
Retrieving Results for Index 259: mount gambier.
Closest hotel in mount gambier is Commodore on the Park.
--------------------Ending Search
Retrieving Results for Index 260: gusau.
Closest hotel in gusau is Crystal Merit Suite and Apartments.
--------------------Ending Search
Retrieving Results for Index 261: anadyr.
Closest hotel in anadyr is Chukotka hotel.
--------------------Ending Search
Retrieving Results for Index 262: touros.
Closest hotel in touros is Bugaendrus Paradise.
--------------------E

Closest hotel in antalaha is Villa-Malaza.
--------------------Ending Search
Retrieving Results for Index 320: cairns.
Closest hotel in cairns is Shangri-La The Marina, Cairns.
--------------------Ending Search
Retrieving Results for Index 321: road town.
Closest hotel in road town is Sea View Hotel.
--------------------Ending Search
Retrieving Results for Index 322: benghazi.
Closest hotel in benghazi is Tibesti Hotel.
--------------------Ending Search
Retrieving Results for Index 323: raga.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 324: kiama.
Closest hotel in kiama is The Sebel Kiama Harbourside.
--------------------Ending Search
Retrieving Results for Index 325: qasigiannguit.
Closest hotel in qasigiannguit is Hotel Diskobay.
--------------------Ending Search
Retrieving Results for Index 326: tome.
Closest hotel in tome is Cabañas Punta De Parra.
--------------------Ending Search
Retrieving Results for Index 327: 

Closest hotel in yar-sale is Hotel "Nord House".
--------------------Ending Search
Retrieving Results for Index 383: sept-iles.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 384: ransang.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 385: melito di porto salvo.
Closest hotel in melito di porto salvo is Hotel Tito Serranò.
--------------------Ending Search
Retrieving Results for Index 386: clyde river.
Closest hotel in clyde river is Qamaq Hotel.
--------------------Ending Search
Retrieving Results for Index 387: pandhana.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 388: cap-aux-meules.
Closest hotel in cap-aux-meules is Château Madelinot.
--------------------Ending Search
Retrieving Results for Index 389: inongo.
Closest hotel in inongo is Hotel Du Lac.
--------------------Ending Search
Retri

Closest hotel in hohoe is Kiki's Court.
--------------------Ending Search
Retrieving Results for Index 446: hobyo.
Closest hotel in hobyo is Osmani Hotel.
--------------------Ending Search
Retrieving Results for Index 447: gornopravdinsk.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 448: hofn.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 449: sabancuy.
Closest hotel in sabancuy is Hotel Aguilar.
--------------------Ending Search
Retrieving Results for Index 450: brae.
Closest hotel in brae is Brae Hotel.
--------------------Ending Search
Retrieving Results for Index 451: najran.
Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 452: bandar-e lengeh.
Closest hotel in bandar-e lengeh is Diplomat Hotel.
--------------------Ending Search
Retrieving Results for Index 453: condega.
Closest hotel in con

Could not find result
-------------------------
--------------------Ending Search
Retrieving Results for Index 510: tiarei.
Closest hotel in tiarei is Le Rocher de Tahiti.
--------------------Ending Search
Retrieving Results for Index 511: eenhana.
Closest hotel in eenhana is Hotel Litu.
--------------------Ending Search
Retrieving Results for Index 512: mae sai.
Closest hotel in mae sai is CAPITAL O 1115 Wang Thong Hotel Maesai.
--------------------Ending Search
Retrieving Results for Index 513: nantucket.
Closest hotel in nantucket is White Elephant Nantucket.
--------------------Ending Search
Retrieving Results for Index 514: grand gaube.
Closest hotel in grand gaube is LUX Grand Gaube Resort & Villas.
--------------------Ending Search
Retrieving Results for Index 515: ust-tsilma.
Closest hotel in ust-tsilma is Sberbank.
--------------------Ending Search
Retrieving Results for Index 516: solnechnyy.
Closest hotel in solnechnyy is Hotel complex Pastoral.
--------------------Ending Se

In [38]:
hotel_df


,City,Country,Lat,Lng,Hotel Name
0,barrow,US,71.2906,-156.7887,Top of the World Hotel
1,pisco,PE,-13.7000,-76.2167,REGENCY PLAZA HOTEL
2,arraial do cabo,BR,-22.9661,-42.0278,Ocean View Hotel
3,kapaa,US,22.0752,-159.3190,Kauai Shores Hotel
4,saint-philippe,RE,-21.3585,55.7679,Les Embruns Du Baril
...,...,...,...,...,...
558,meulaboh,ID,4.1363,96.1285,Bin Daod Hotel and Restaurant
559,coffs harbour,AU,-30.3000,153.1333,The Pier Hotel
560,muyuka,CM,4.2898,9.4103,
561,contamana,PE,-7.3333,-75.0167,Hotel Mechita


In [39]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [40]:
# Add marker layer ontop of heat map
# Display figure
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)
fig




Figure(layout=FigureLayout(height='420px'))